# <center>Deep Generative Models</center>
## <center>Seminar 9</center>

<center>24.04.2025</center>

## DDPM

<center><img src="pics/diffusion_pgm.png" width=1000 /></center>

Diffusion models are latent variable models of the form
$$
p_\theta(x_0) := \int p_\theta(x_0 | z) p_\theta(z) dz,
$$
where $z = (x_1, \dots, x_T)$ are latents of the same dimensionality as the data $x_0 \sim q(x_0)$.


## **Qustion 1.** How we can define encoder in such VAE?

In this case, the most convenient way to build an encoder is to fix a Markov chain that gradually adds Gaussian noise to the data, according to  
$$
    q(z | x) = q(x_1, \dots, x_T | x_0) = \prod_{t = 1}^T q(x_t | x_{t - 1}),
$$
where
$$
q(x_t \mid x_{t-1}) := \mathcal{N}\left(x_t; \sqrt{1 - \beta_t} \, x_{t-1}, \beta_t \, \mathbf{I}\right)
$$

**Sub-question:**  
Why do we need to add a Gaussian forward process?

1. $q(x_{\infty} \mid x_0) = \mathcal{N}(0, I)$ — by repeatedly multiplying by a factor $0 < \sqrt{1 - \beta} < 1$, we progressively blur the original image.  
2. $p_{\infty}(x) = \mathcal{N}(0, I)$ — the mean is driven to zero, and we inject a large amount of Gaussian noise — resulting in pure Gaussian noise.  

## **Qustion 2.** Why do we need Markov property?

The Markov property of $q(x_t \mid x_{t-1})$ is mainly due to the simplicity it provides in constructing the forward process. Now, we have an analytic formula that directly applies noise to the image from $x_0$ to $x_t$:
$$
q(x_t \mid x_0) = \mathcal{N}\left(x_t; \sqrt{\bar{\alpha}_t} \, x_0, (1 - \bar{\alpha}_t) \, \mathbf{I} \right),
$$
where $\bar{\alpha}_t = \prod_{s=1}^t (1 - \beta_s)$.

## **Qustion 3.** How would we train such "VAE"?

Of course it is ELBO:

$$
\mathbb{E} \left[ -\log p_\theta(\mathbf{x}_0) \right] 
\leq \mathbb{E}_q \left[ 
    -\log \frac{p_\theta(\mathbf{x}_{0:T})}{q(\mathbf{x}_{1:T} \mid \mathbf{x}_0)} 
\right] 
= 
$$

$$
=\mathbb{E}_q \left[
D_{\mathrm{KL}}(q(\mathbf{x}_T \mid \mathbf{x}_0) \,\|\, p(\mathbf{x}_T))
+ \sum_{t > 1} D_{\mathrm{KL}}(q(\mathbf{x}_{t-1} \mid \mathbf{x}_t, \mathbf{x}_0) \,\|\, p_\theta(\mathbf{x}_{t-1} \mid \mathbf{x}_t))
- \log p_\theta(\mathbf{x}_0 \mid \mathbf{x}_1)
\right]
$$

So, what we actually need to minimize is the KL divergence between the true posterior of the forward process and the learnable reverse process:

$$
\text{KL}\left(q(\mathbf{x}_{t-1} \mid \mathbf{x}_t, \mathbf{x}_0) \,\|\, p_\theta(\mathbf{x}_{t-1} \mid \mathbf{x}_t)\right),
$$

where the posterior $q(\mathbf{x}_{t-1} \mid \mathbf{x}_t, \mathbf{x}_0)$ can be derived analytically using Bayes’ rule, since all involved distributions are Gaussian:

$$
q(\mathbf{x}_{t-1} \mid \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}(\mathbf{x}_{t-1}; \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \mathbf{x}_0 + \frac{\sqrt{1 - \beta_t} (1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{x}_t, \frac{(1 - \bar{\alpha}_{t-1}) \beta_t}{1 - \bar{\alpha}_t} \mathbf{I}),
$$


## **Qustion 4.** How to parametrize $p_\theta(\mathbf{x}_{t-1} \mid \mathbf{x}_t)$?

We define the reverse transition as a Gaussian:
$$
p_\theta(\mathbf{x}_{t-1} \mid \mathbf{x}_t) = \mathcal{N}(\mathbf{x}_{t-1}; \mu_\theta(\mathbf{x}_t, t), \Sigma_\theta(\mathbf{x}_t, t))
$$

**Mean $\mu_\theta$** is predicted by a neural network conditioned on $\mathbf{x}_t$ and timestep $t$

**Variance $\Sigma_\theta$** is fixed to a diagonal matrix $\sigma_t^2 \mathbf{I}$ with $\sigma_t$ having two choises:
1. **$ \sigma_t^2 = \beta_t $**  
   - This matches the **forward process noise** at each step.  
   - **Optimal when** the data $ \mathbf{x}_0 $ is assumed to follow a standard Gaussian: $$ \mathbf{x}_0 \sim \mathcal{N}(0, \mathbf{I}) $$
   - Leads to **higher entropy** (more uncertainty) in the reverse step.
1. **$ \sigma_t^2 = \tilde{\beta}_t = \frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t $**  
   - Derived from the posterior of the forward process.  
   - **Optimal when** $ \mathbf{x}_0 $ is **fixed** (i.e., no uncertainty).  
   - Produces **lower entropy**, more confident reverse predictions.


## **Qustion 5.** How we can simplify it?

Indeed sinse we know KL between Normal distribtuions and $\sigma$ is fixed we can just:

$$
L_{t-1} = \mathbb{E}_q \left[
\frac{1}{2\sigma_t^2} \left\| \tilde{\mu}_t(\mathbf{x}_t, \mathbf{x}_0) - \mu_\theta(\mathbf{x}_t, t) \right\|^2
\right]
= \mathbb{E}_{\mathbf{x}_0, \epsilon} \left[
\frac{1}{2\sigma_t^2} \left\|
\frac{1}{\sqrt{\alpha_t}} \left( \mathbf{x}_t(\mathbf{x}_0, \epsilon) - \frac{\beta_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon \right) 
- \mu_\theta(\mathbf{x}_t(\mathbf{x}_0, \epsilon), t)
\right\|^2
\right],
$$

where $\mathbf{x}_t(\mathbf{x}_0, \epsilon) = \sqrt{\bar{\alpha}_t} \, \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_t} \, \epsilon, \quad \epsilon \sim \mathcal{N}(0, \mathbf{I})
$

Since $ \mathbf{x}_t(\mathbf{x}_0, \epsilon) $ is available, we reparameterize the reverse mean as:

$$
\mu_\theta(\mathbf{x}_t, t) = \frac{1}{\sqrt{\alpha_t}} \left( \mathbf{x}_t(\mathbf{x}_0, \epsilon) - \frac{\beta_t}{\sqrt{1 - \bar{\alpha}_t}} \, \epsilon_\theta(\mathbf{x}_t, t) \right)
$$

where $ \epsilon_\theta(\mathbf{x}_t, t) $ predicts the noise $ \epsilon $ added during the forward process.

Instead of matching the posterior mean, we directly train $ \epsilon_\theta $ to predict noise using the following loss:

$$
\mathbb{E}_{\mathbf{x}_0, \epsilon} \left[
\frac{\beta_t^2}{2 \sigma_t^2 \alpha_t (1 - \bar{\alpha}_t)} 
\left\| \epsilon - \epsilon_\theta(\mathbf{x}_t, t) \right\|^2
\right] = \mathbb{E}_{\mathbf{x}_0, \epsilon} \left[ 
\left\| \epsilon - \epsilon_\theta(\mathbf{x}_t, t) \right\|^2
\right]
$$

## **Qustion 6.** Why do we need to make small steps to have good quality of samples?

How do we sample at first:

$$
x_{t-1} =
\frac{1}{\sqrt{1 - \beta_t}} 
\left(
x_t - \frac{\beta_t}{\sqrt{1 - \bar{\alpha}_t}} \, \epsilon_\theta(x_t, t)
\right)
+
\sqrt{\beta_t} z
$$


Remember initially we have Gaussian assumption:
$$
p(x_{t-1} \mid x_t) \approx \mathcal{N}(\mu_\theta(x_t, t), \sigma_t^2 \mathbf{I})
$$

Lets look at fromula from lecture that follows from Bayes’ rule:
$$
p(x_{t-1} \mid x_t) = \frac{q(x_t \mid x_{t-1}) \, q(x_{t-1})}{q(x_t)}
$$

If the noise added in each step is **small** (small $ \beta_t $), the distributions are **locally smooth** and the log-density can be approximated by a **second-order Taylor expansion** resulting in a **Gaussian** reverse kernel. See **Feller theorem**.

## **Qustion 7.** How to make DDPM faster?
<center><img src="pics/ddim.png" width=1000 /></center>

To increaset the speed lets consider non-Markov forward process (encoder):

$$
q_\sigma(x_{1:T} \mid x_0) := q_\sigma(x_T \mid x_0) \prod_{t=2}^T q_\sigma(x_{t-1} \mid x_t, x_0),
$$

where terminal distribution is $q_\sigma(x_T \mid x_0) = \mathcal{N}\left(\sqrt{\bar{\alpha}_T} x_0, (1 - \bar{\alpha}_T) \mathbf{I} \right),$
and for all $ t > 1 $, the transitions are:
$$
q_\sigma(x_{t-1} \mid x_t, x_0) = \mathcal{N} \left(
\sqrt{\bar{\alpha}_{t-1}} x_0 + \sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot \frac{x_t - \sqrt{\bar{\alpha}_t} x_0}{\sqrt{1 - \bar{\alpha}_t}},
\; \sigma_t^2 \, \mathbf{I}
\right)
$$
**Note:**  
The mean function in $ q_\sigma(x_{t-1} \mid x_t, x_0) $ is carefully chosen to ensure that the marginal distribution remains consistent with the original forward process:
$$
q_\sigma(x_t \mid x_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_t} \, x_0, (1 - \bar{\alpha}_t) \, \mathbf{I} \right)
$$

To minimize the they similarly derive ELBO:

$$
\mathbb{E}_{q_\sigma} \left[
\log q_\sigma(x_T \mid x_0) + \sum_{t=2}^{T} \log q_\sigma(x_{t-1} \mid x_t, x_0)
- \sum_{t=1}^{T} \log p_\theta^{(t)}(x_{t-1} \mid x_t)
- \log p_\theta(x_T)
\right]
$$

In the same manner they derive loss in noise prediction manner. Moreover, authors prooved **this loss is eqivalent to DDPM loss**, which means we can use previously train DDPM to apply following sampling scheme:

$$
x_{t-1} = 
\sqrt{\bar{\alpha}_{t-1}} 
\left(
\frac{x_t - \sqrt{1 - \bar{\alpha}_t} \epsilon_\theta^{(t)}(x_t)}{\sqrt{\bar{\alpha}_t}}
\right)
+
\sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot \epsilon_\theta^{(t)}(x_t)
+
\sigma_t \epsilon_t
$$

**Note:** when the noise schedule is set to $\sigma_t = \sqrt{\frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t}} \cdot \sqrt{1 - \frac{\bar{\alpha}_t}{\bar{\alpha}_{t-1}}},$
for all $t$, the forward process becomes **Markovian**, and the overall generative model reduces to a **DDPM**.

